# Fine tune BlenderBot 

In [1]:
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration, Trainer, TrainingArguments

c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


### Load model and run 

In [2]:
tokenizer = BlenderbotTokenizer.from_pretrained("facebook/blenderbot-400M-distill")
model = BlenderbotForConditionalGeneration.from_pretrained("facebook/blenderbot-400M-distill")

c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
#making an utterance
utterance = "My name is gold, I like football and coding"
#tokenize the utterance
inputs = tokenizer(utterance, return_tensors="pt")
#generate model results
result = model.generate(**inputs)
# result
tokenizer.decode(result[0])

'<s> Nice to meet you, my name is john. What do you do for a living?</s>'

### Load, extract and processing data

data được tải về từ https://www.kaggle.com/datasets/grafstor/simple-dialogs-for-chatbot?resource=download

In [4]:
f = open('data/dialogs.txt','r')
file = f.read()
conversations = file.split('\n')

In [5]:
data = []
for i in conversations:
    question, anwser = i.split("\t")
    item = {
        "input": question,
        "respone": anwser
    }
    data.append(item)

print('number of conversation:', len(data))

number of conversation: 3725


In [6]:
import json

with open('data/data.json','w', encoding='utf-8') as f:
    json.dump(data,f,ensure_ascii=False, indent=4)

### Tokenize data

In [7]:
from datasets import load_dataset

dataset = load_dataset('json', data_files='data/data.json')

# Split the dataset into training and validation sets
train_test_split = dataset['train'].train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
valid_dataset = train_test_split['test']

Extracting data files: 100%|██████████| 1/1 [00:00<00:00,  5.97it/s]
Generating train split: 3725 examples [00:00, 4071.93 examples/s]


In [12]:
train_dataset

Dataset({
    features: ['input', 'respone', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3352
})

In [8]:
def tokenize_function(examples):
    inputs = examples['input']
    responses = examples['respone']
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length')

    # Tokenize the responses with the `text_target` keyword argument
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(responses, max_length=128, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
tokenize_function(data[0])

c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [7336, 19, 544, 366, 304, 929, 38, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1], 'labels': [607, 476, 1435, 21, 544, 458, 1498, 38, 2]}

In [10]:
# Tokenize the dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 373/373 [00:00<00:00, 3553.38 examples/s]


### Fine tunning model

In [11]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)


# Fine-tune the model
trainer.train()

TypeError: Accelerator.__init__() got an unexpected keyword argument 'use_seedable_sampler'

### share model

In [ ]:
model_name = 'facebook/blenderbot-400M-distill'

kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned",
    "finetuned_from": model_name,
    "tasks": "audio-classification",
}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub(**kwargs)